In [1]:
import pandas as pd
import os
# read excel file need pip install openpyxl

## 計算出平均有多少顧客在銀行裡面
- Arrival time : 到達時間
- X1：等待時間
- X2：服務時間
- X3：等待時間＋服務時間＝離開時間

In [2]:
df = pd.read_excel("./Supplementary DATA on Queues/FIRST BANK (WEEK 1).xlsx", sheet_name="Sheet1")
df = df.iloc[:, :5]

# 移除奇怪的值 => 10,033
df = df.drop(266)
df["Arrival time"] = df["Arrival time"].astype("str").str.replace(".", ":")
df["Arrival time"] = pd.to_datetime(df["Arrival time"], format='%H:%M').dt.strftime("%H:%M")

df["Start Service time"] = pd.to_datetime(df["Arrival time"]) + pd.to_timedelta(df["X2"], unit="m")
df["Start Service time"] = df["Start Service time"].dt.strftime("%H:%M")

df["Leave time"] = pd.to_datetime(df["Arrival time"]) + pd.to_timedelta(df["X3"], unit="m")
df["Leave time"] = df["Leave time"].dt.strftime("%H:%M")

df = df[['Number', 'Arrival time', 'X1', 'X2', 'Start Service time', 'X3', 'Leave time']]

df

/var/folders/vc/svw_78hn1n79nnpst2w2mmyc0000gn/T/ipykernel_1297/2989676494.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Arrival time"] = df["Arrival time"].astype("str").str.replace(".", ":")


,Number,Arrival time,X1,X2,Start Service time,X3,Leave time
0,1,08:00,11,28,08:28,39,08:39
1,2,08:00,13,25,08:25,38,08:38
2,3,08:00,14,26,08:26,40,08:40
3,4,08:00,9,26,08:26,35,08:35
4,5,08:00,14,23,08:23,37,08:37
...,...,...,...,...,...,...,...
875,876,14:58,11,23,15:21,34,15:32
876,877,14:59,15,27,15:26,42,15:41
877,878,14:59,14,27,15:26,41,15:40
878,879,14:59,13,29,15:28,42,15:41


In [3]:
start_time = df["Arrival time"].min()
end_time = df["Leave time"].max()
print(start_time, end_time)

08:00 15:41


In [4]:
time_points = pd.date_range(start_time, end_time, freq="1min").strftime('%H:%M')
time_points

customer_count = {time_point: 0 for time_point in time_points}

for time in time_points:
    customer_count[time] = df[(df["Arrival time"] <= time) & (df["Leave time"] > time)].shape[0]

result = pd.DataFrame(customer_count.items(), columns=["Time", "Customer Count"])
result

,Time,Customer Count
0,08:00,8
1,08:01,19
2,08:02,21
3,08:03,23
4,08:04,31
...,...,...
457,15:37,5
458,15:38,4
459,15:39,3
460,15:40,2


In [5]:
average_customer_count = result["Customer Count"].mean()
print(f"平均下來，以一分鐘為單位，同一分鐘內有 {int(average_customer_count)} 人在銀行處理業務")

平均下來，以一分鐘為單位，同一分鐘內有 72 人在銀行處理業務


In [6]:
folder_path = "./Supplementary DATA on Queues"

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(file_path)

./Supplementary DATA on Queues/THIRD BANK (WEEK 2).xlsx
./Supplementary DATA on Queues/SECOND  BANK (WEEK 1).xlsx
./Supplementary DATA on Queues/THIRD BANK (WEEK 4).xlsx
./Supplementary DATA on Queues/SECOND  BANK (WEEK 3).xlsx
./Supplementary DATA on Queues/THIRD BANK (WEEK 3).xlsx
./Supplementary DATA on Queues/SECOND  BANK (WEEK 4).xlsx
./Supplementary DATA on Queues/THIRD BANK (WEEK 1).xlsx
./Supplementary DATA on Queues/SECOND  BANK (WEEK 2).xlsx
./Supplementary DATA on Queues/FIRST BANK (WEEK 3).xlsx
./Supplementary DATA on Queues/FIRST BANK (WEEK 1).xlsx
./Supplementary DATA on Queues/FIRST BANK (WEEK 4).xlsx
./Supplementary DATA on Queues/FIRST BANK (WEEK 2).xlsx


### 發現前處理花了很多時間
- 時間格式不一，有些是用 "." 區隔，還要轉格式
- 部分時間還與其他時間有出入 （e.g. 大家都是 10.03 / 14.25 這樣，有時間卻長這樣： 10,033）
- 有些資料表使用 24小時制 有些使用 12小時制

其他檔案大致也都是前處理得當，再來計算即可，就不做了！